# Streaming流式响应提升用户体验

对于LLMs，流式处理已成为一项越来越受欢迎的功能。Streaming是在开始生成token时快速返回，而不是等待创建完整响应后再返回所有的内容，这样也可以模拟真人对话的节奏感，让用户感觉不那么"机器人"。就像你体验过的GPT、Claude.ai、GLM等AI能力一样，它们在用户提问后，都采用了打字机形式的回复方式，让用户感受到数据在不断流入屏幕，从而提高了聊天的流畅性，**并极大地加快了首次回应给用户的响应时间**，即Time to First Token (TTFT)。

1. **Streaming流式响应的作用:**
   - 逐个令牌(token)流式传输响应
   - 用户可以实时看到响应的生成过程

2. **Streaming流式响应的优点:**
   - 避免长时间等待完整响应
   - 提高效率和交互性
   - 使对话感觉更自然，更像人类思考过程

### 服务端如何发送Streaming响应

在需要实时数据的场景下，传统的HTTP请求-响应模型存在一些局限性——客户端主动发请求，服务端被动地返回响应的单向通信模式。这种模型在客户端需要实时获取结果的场景下是不合适的，因为这意味着客户端需要不断地轮询。所以最好的做法是服务端生成结果之后，主动推送给客户端。

可能有小伙伴知道WebSocket，WebSocket是一种双向通信协议，允许服务器和客户端之间建立持久连接，双方都可以主动发送数据，常用于实时聊天应用、在线游戏中。但选用它有些太重了，因为这意味着要从HTTP切换到WebSocket协议，而且它是双向通信的，而我们只需要接收就可以了。

有没有办法，我们仍然使用HTTP协议，同时还能让服务端主动推送数据呢？

铛铛铛铛～ SSE技术来了，它的英文全称是Server-Sent Events（服务端推送事件）。通过SSE可以让服务端即时推送数据到客户端，而不需要客户端轮询服务端以获取更新。

GLM-4的API也支持通过SSE进行流式响应。当`stream`参数设置为`True`时，模型将通过标准Event Stream逐块返回生成的内容。当Event Stream结束时，将返回一个`data: [DONE]`消息。

流式传输就像用水管接水一样，数据不是一次性全部传输完成的，而是像水流一样持续传输。这样做的优势在于：

- **实时性强**：可以更快地获取到第一批数据，不用等到所有数据都传输完毕。
- **节省内存**：不需要将所有数据都存储在内存中，可以边接收边处理。
- **处理大数据**：对于非常大的数据，流式传输可以避免内存溢出。

虽然这种方式需要对数据进行解析和状态管理，但不用担心，我们导入的`zhipuai`库已经帮助我们处理了SSE数据流，并将其转换为Python对象，方便用户处理。

### 如何使用Streaming功能

既然`zhipuai`库已经帮助我们处理了服务器返回的原始数据并转换为用户友好的Python消息对象，那么对于我们来说，想要使用Streaming的功能，只需要开启它，并解析流式响应的结果即可。

以下是具体的操作步骤：

1. **导入`ZhipuAI`库并创建客户端实例：**

In [1]:
from zhipuai import ZhipuAI

# 请填写您自己的API Key
# client = ZhipuAI(api_key="your_api_key_here")
client = ZhipuAI()

2. **发送请求并设置`stream=True`：**

要从API获取流式响应，只需将`stream=True`传递给`client.chat.completions.create`即可。

In [4]:
stream = client.chat.completions.create(
    model="glm-4-plus",
    messages=[
        {
            "role": "user",
            "content": "为什么空调能吹出冷热两种不同的风？",
        }
    ],
    stream=True
)
type(stream)

zhipuai.core._sse_client.StreamResponse

3. **处理流式响应：**

`stream`看起来是一个对象，但因为它实现了Python的迭代器协议，这使得它可以被用在`for`循环中。我们可以逐个处理返回的`chunk`。

具体来说：
1. `Stream` 对象实现了 `__iter__()` 和 `__next__()` 方法
2. 这使得它成为一个可迭代对象（iterable）
3. 每次调用 `__next__()` 会返回流中的下一个数据块（chunk）所以我们可以看一下chunk的结构来提取内容

简单代码的话，就像这样：
```python
class Stream:
    def __iter__(self):
        return self
        
    def __next__(self):
        # 获取下一个数据块
        if has_more_data:
            return next_chunk
        else:
            raise StopIteration

# 所以可以这样使用
for chunk in stream:
    process(chunk)
```

In [7]:
stream = client.chat.completions.create(
    model="glm-4-plus",
    messages=[
        {
            "role": "user",
            "content": "讲几个冷知识",
        }
    ],
    stream=True
)
# 处理流式响应
for chunk in stream:
    print(chunk)

ChatCompletionChunk(id='2024111214504600b25dd690da494a', choices=[Choice(delta=ChoiceDelta(content='当然', role='assistant', tool_calls=None), finish_reason=None, index=0)], created=1731394246, model='glm-4-plus', usage=None, extra_json=None)
ChatCompletionChunk(id='2024111214504600b25dd690da494a', choices=[Choice(delta=ChoiceDelta(content='可以', role='assistant', tool_calls=None), finish_reason=None, index=0)], created=1731394246, model='glm-4-plus', usage=None, extra_json=None)
ChatCompletionChunk(id='2024111214504600b25dd690da494a', choices=[Choice(delta=ChoiceDelta(content='！', role='assistant', tool_calls=None), finish_reason=None, index=0)], created=1731394246, model='glm-4-plus', usage=None, extra_json=None)
ChatCompletionChunk(id='2024111214504600b25dd690da494a', choices=[Choice(delta=ChoiceDelta(content='以下', role='assistant', tool_calls=None), finish_reason=None, index=0)], created=1731394246, model='glm-4-plus', usage=None, extra_json=None)
ChatCompletionChunk(id='2024111214504


可以看到，数据流是一系列 `ChatCompletionChunk` 对象，每个对象包含以下属性：

- **id**: 唯一标识符
- **choices**: 一个包含 `Choice` 对象的列表，每个 `Choice` 对象包含 `delta`（即新生成的内容）、`role`、`tool_calls` 等信息
- **created**: 创建时间戳
- **model**: 使用的模型名称
- **usage**: 令牌使用情况（在大多数情况下为 `None`，只有最后一个块包含实际的使用数据）
- **extra_json**: 额外的 JSON 数据

如果我们只想提取`chunk`中的内容部分，可以这样操作：

In [3]:
stream = client.chat.completions.create(
    model="glm-4-plus",
    messages=[
        {
            "role": "user",
            "content": "有透明的金属吗？",
        }
    ],
    stream=True
)
# 处理流式响应
for chunk in stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content)

透明的
金属
听起来
像
是一个
矛盾
的概念
，
因为
金属
通常
以其
不
透明
和
反射
光的
特性
而
著称
。
然而
，
科学
研究和
材料
技术的发展
确实
已经
创造出
了一些
具有
透明
特性的
金属
基
材料
。

这些
透明
金属
通常
是通过
以下
几种
方式
实现的
：

1
.
 **
纳米
结构
材料
**
：
通过
将
金属
结构
缩小
到
纳米
尺度
，
可以
改变
其
光学
性质
，
使得
某些
金属
（
如
金
、
银
）
在某些
特定
条件下
表现出
透明
性
。

2
.
 **
金属
网格
**
：
通过
将
金属
制成
极
细
的
网格
结构
，
可以使
光线
通过
，
从而
实现
一定
程度的
透明
性
。

3
.
 **
合金
和
复合材料
**
：
通过
将
金属
与其他
材料
（
如
氧化物
）
结合
，
可以
创造出
具有
透明
特性的
合金
或
复合材料
。

4
.
 **
离子
注入
**
：
通过
向
金属
中
注入
其他
元素的
离子
，
可以
改变
其
光学
性质
，
使其
变得
透明
。

一个
著名的
例子
是
**
氧化
铟
锡
（
ITO
）
**
，
它
是一种
广泛
用于
触摸
屏
和
显示器
中的
透明
导电
材料
。
虽然
它
不是
纯
金属
，
但它
含有
金属
元素
，
并且在
可见
光
范围内
表现出
良好的
透明
性
。

需要注意的是
，
这些
透明
金属
或
类
金属
材料的
透明
度
通常
不如
玻璃
等
传统
透明
材料
，
它们
往往
在
特定
波长
范围内
透明
，
并且
可能
具有
特殊的
用途
，
比如
在
电子
设备
、
光学
仪器
和
高级
窗口
材料
中
。

总体
而言
，
虽然
完全
像
玻璃
一样
透明的
纯
金属
还不
存在
，
但
通过各种
技术和
材料
科学
手段
，
我们已经
能够
制备
出
具有一定
透明
度的
金属
基
材料
。


### 实时输出与缓冲区管理

在Python中，`print()`函数的默认行为是在行末自动添加一个换行符（`\n`）。为了在输出逐渐到达时保持实时更新，并且将输出显示在同一行，以便更清晰地查看完整的响应，在处理实时流数据时，通常会设置`end=""`和`flush=True`。

In [4]:
stream = client.chat.completions.create(
    model="glm-4-plus",
    messages=[
        {
            "role": "user",
            "content": "讲几个冷知识",
        }
    ],
    stream=True
)
for chunk in stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

当然可以！以下是一些有趣的冷知识：

1. **蜜蜂的飞行原理**：
   蜜蜂的翅膀太小，按照常规空气动力学原理，它们 eigentlich（实际上）不应该能飞。但蜜蜂通过快速拍打翅膀，制造出涡流，从而获得升力。

2. **章鱼的血液是蓝色的**：
   章鱼的血液中含有一种叫做血蓝蛋白的蛋白质，这种蛋白质含有铜，使得它们的血液呈现蓝色。

3. **太空中的火焰是球形的**：
   在地球上，火焰通常是锥形的，因为重力会影响热空气的上升。但在太空的失重环境中，火焰因为没有重力的影响，会呈现球形。

4. **企鹅求偶石**：
   在企鹅的社会中，雄性企鹅会精心挑选光滑的石头送给心仪的雌性企鹅。如果雌性企鹅接受了这块石头，两只企鹅就会成为伴侣并共同搭建巢穴。

5. **人类的鼻子可以记住5万种不同的气味**：
   虽然我们的嗅觉不如某些动物灵敏，但人类的鼻子仍然能够分辨和记住多达5万种不同的气味。

6. **香蕉是浆果，草莓不是**：
   在植物学上，香蕉被归类为浆果，而草莓则属于聚合果。这是因为香蕉是由单个花的单个子房发育而来的，而草莓则是由多个花的多个子房发育而来的。

7. **海星没有大脑**：
   海星没有中枢神经系统，它们通过一个复杂的神经网络来协调身体的活动。

8. **世界上最古老的持续经营的公司是一家日本建筑公司**：
   这家公司叫做金刚组（Kongo Gumi），成立于公元578年，至今已有1400多年的历史，专门从事寺庙建筑。

9. **蜗牛的牙齿长在舌头上**：
   蜗牛的舌头上布满了细小的牙齿，它们用这些牙齿来刮取食物。

10. **地球上的水循环系统**：
    地球上的水循环系统非常高效，平均每15亿年，地球上的水就会完成一次完整的循环。

希望这些冷知识能让你感到新奇有趣！

### 获取token使用情况

令牌使用情况在大多数情况下为 `None`，只有最后一个块包含实际的使用数据,所以我们可以在处理每个块时检查 usage 属性。当 usage 不为 None 时，表示这是最后一个块，提取token使用情况



In [10]:
stream = client.chat.completions.create(
    model="glm-4-plus",
    messages=[
        {
            "role": "user",
            "content": "根据热力学第二定律，世界将越来越混乱。那为什么会产生能体现秩序的细胞、生物和人类？",
        }
    ],
    stream=True,
)

def process_stream(stream):
    usage_info = None
    for chunk in stream:
        # 处理内容
        if chunk.choices and chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end='', flush=True)
        
        # 检查 usage
        if chunk.usage:
            usage_info = chunk.usage
    
    if usage_info:
        print(f"\nPrompt Tokens: {usage_info.prompt_tokens}")
        print(f"Completion Tokens: {usage_info.completion_tokens}")
        print(f"Total Tokens: {usage_info.total_tokens}")
    else:
        print("\n未找到 usage 信息。")

process_stream(stream)

热力学第二定律确实指出，一个孤立系统的总熵（即无序度）随时间推移会增加，意味着系统会趋向于更加混乱的状态。然而，这一规律并不排斥在局部范围内出现秩序的增加，尤其是在非孤立系统中。以下是几个关键点来解释为什么会产生体现秩序的细胞、生物和人类：

1. **能量输入**：地球不是一个孤立系统，它持续从太阳接收能量。太阳能为地球上的生物提供了必要的能量，使得局部的熵减少成为可能。例如，植物通过光合作用将太阳能转化为化学能，建立了有序的生物分子结构。

2. **开放系统**：生物体是开放系统，它们与外界环境进行物质和能量的交换。在这种交换过程中，生物体可以通过消耗外界能量来维持或增加自身的秩序，即使这会导致其周围环境的熵增加。

3. **自组织现象**：在合适的条件下，自然界中可以出现自组织现象，即系统在没有外部特定指令的情况下自发形成有序结构。例如，某些化学反应可以形成复杂的空间和时间结构，这种现象在生物形成过程中也起到重要作用。

4. **自然选择和进化**：生物进化通过自然选择机制，使得那些更有序、更适应环境的生物体得以生存和繁衍。这种过程逐步累积，导致了复杂生物结构的出现，包括细胞、多细胞生物乃至人类。

5. **局部熵减与全局熵增**：在局部范围内（如生物体内），熵可以减少，形成高度有序的结构，但这通常伴随着环境中熵的增加。总体来看，宇宙的总熵还是在增加的。

因此，虽然热力学第二定律预言了孤立系统无序度的增加，但通过能量输入、开放系统特性、自组织现象以及自然选择和进化等机制，局部范围内的秩序增加是完全可能的，这解释了细胞、生物和人类等复杂有序结构的出现。
Prompt Tokens: 29
Completion Tokens: 379
Total Tokens: 408


### 流式处理的时间优势

流式处理能够显著减少用户等待的时间，提高用户体验。以下是一个对比非流式处理和流式处理的示例：

**非流式处理方法：**

In [5]:
import time

def measure_non_streaming_ttft():
    start_time = time.time()

    response = client.chat.completions.create(
        model="glm-4-plus",
        messages=[
            {
                "role": "user",
                "content": "写一篇长篇文章，解释唐朝的历史",
            }
        ]
    )

    response_time = time.time() - start_time

    print(response.choices[0].message.content)
    print(f"接收第一个token的时间: {response_time:.3f}秒")
    print(f"接收完整响应的时间: {response_time:.3f}秒")
    print(f"生成的总token数: {response.usage.total_tokens}")


**流式处理方法：**

In [6]:
import time

def measure_streaming_ttft():
    start_time = time.time()

    stream = client.chat.completions.create(
        model="glm-4-plus",
        messages=[
            {
                "role": "user",
                "content": "写一篇长篇文章，解释唐朝的历史",
            }
        ],
        stream=True,
    )

    have_received_first_token = False
    for chunk in stream:
        if chunk.choices[0].delta.content:
            if not have_received_first_token:
                ttft = time.time() - start_time
                have_received_first_token = True
            content_chunk = chunk.choices[0].delta.content
            print(content_chunk, end='', flush=True)
        if chunk.usage:
            total_tokens = chunk.usage.total_tokens

    total_time = time.time() - start_time
    print(f"\n接收第一个token的时间: {ttft:.3f}秒", flush=True)
    print(f"接收完整响应的时间: {total_time:.3f}秒", flush=True)
    print(f"生成的总token数: {total_tokens}", flush=True)



In [7]:
measure_non_streaming_ttft()

**唐朝：中华文明的黄金时代**

在中国悠久的历史长河中，唐朝无疑是一颗璀璨的明珠，其辉煌的成就不仅在中国历史上留下了深刻的印记，而且对世界文明也产生了深远的影响。唐朝（618-907年），是中国封建社会的一个高峰，以其开放包容的文化、繁荣的经济、强大的军事和先进的制度，被后世誉为“盛唐”。

**一、唐朝的建立与初期发展**

唐朝的建立者是李渊，他在隋朝末年趁乱起兵，最终推翻了隋朝，于公元618年建立了唐朝，定都长安。李渊即位为唐高祖，开启了唐朝近三百年的历史篇章。

唐太宗李世民是唐朝的第二位皇帝，他在位期间（626-649年），通过“玄武门之变”巩固了皇权，推行了一系列改革措施，如减轻赋税、选拔贤能、修订法律等，使得国家迅速恢复和发展，史称“贞观之治”。这一时期，唐朝的政治清明、社会安定、经济繁荣，为后来的盛世奠定了坚实的基础。

**二、唐朝的鼎盛时期**

唐太宗之后，唐高宗李治继位，虽然他在位期间政绩平平，但其皇后武则天却是一位杰出的政治家。武则天在唐高宗晚年和唐中宗时期逐渐掌握实权，最终自立为帝，改国号为周，成为中国历史上唯一的女皇帝。武则天在位期间（690-705年），继续推行唐太宗的政策，并注重发展农业、选拔人才，使得国力进一步增强。

唐朝的鼎盛时期是在唐玄宗李隆基统治下的“开元盛世”（713-741年）。这一时期，唐朝的政治、经济、文化、军事都达到了前所未有的高度。政治上，唐玄宗任用贤能，如姚崇、宋璟等名臣，推行一系列改革措施，提高了行政效率；经济上，农业、手工业、商业都得到了极大的发展，国库充盈；文化上，唐诗达到了巅峰，出现了李白、杜甫等伟大的诗人；军事上，唐朝的疆域达到了最大，势力范围远及中亚。

**三、唐朝的中衰与复兴**

然而，盛极而衰是历史的规律。唐玄宗晚年沉迷于享乐，宠信杨贵妃及其家族，导致朝政腐败，最终引发了“安史之乱”（755-763年）。这场持续了八年的叛乱给唐朝带来了巨大的打击，国力大幅衰退，疆域缩小，中央集权也受到了严重削弱。

安史之乱后，唐朝进入了中衰时期，但也有一些皇帝试图复兴国家。如唐宪宗在位期间（806-820年），通过削弱藩镇势力、整顿财政等措施，使得国家有所恢复，史称“元和中兴”。然而，这些努力并未能从根本上扭转唐朝的衰势。

**四、唐朝的灭亡**

唐朝晚期，政治腐败、藩镇割据、农民起义等问题交织

In [8]:
measure_streaming_ttft()

**唐朝：中华文明的辉煌篇章**

在中国悠久的历史长河中，唐朝无疑是最为璀璨夺目的时期之一。这个历时近三百年的王朝，不仅在中国历史上留下了深刻的印记，也对世界文明产生了深远的影响。本文将从政治、经济、文化、外交等多个角度，全面解析唐朝的辉煌历史。

**一、唐朝的建立与政治制度**

唐朝的建立始于公元618年，李渊在太原起兵反隋，最终建立了唐朝，定都长安，开启了长达289年的统治。唐朝的政治制度在很大程度上继承了隋朝，并在此基础上进行了创新和完善。

唐朝实行的是中央集权制度，皇帝拥有至高无上的权力。中央政府设有三省六部，三省即中书省、门下省、尚书省，分别负责起草诏令、审核诏令和执行诏令，形成了一个严密的政治运作体系。六部则包括吏部、户部、礼部、兵部、刑部、工部，各司其职，管理国家各项事务。

此外，唐朝还实行了科举制度，通过考试选拔人才，这一制度极大地促进了社会流动，为唐朝的繁荣提供了人才保障。

**二、经济繁荣与社会发展**

唐朝的经济繁荣达到了前所未有的高度。农业方面，由于水利设施的完善和农业生产技术的进步，粮食产量大幅增加，为人口增长和社会稳定奠定了基础。手工业方面，丝织、陶瓷、冶金等行业都取得了显著成就，唐三彩更是成为后世传颂的艺术珍品。

商业贸易也十分发达，长安和洛阳成为当时世界上最大的城市之一，吸引了大量外国商人和使节。唐朝还实行了开放的对外贸易政策，通过丝绸之路与西域、中亚乃至欧洲进行广泛的贸易往来。

社会方面，唐朝实行了较为宽松的社会政策，人民生活水平普遍提高。同时，唐朝的社会风气开放，女性地位相对较高，这在当时的封建社会中是难能可贵的。

**三、文化艺术的辉煌成就**

唐朝是中国文化史上的一个高峰时期。文学方面，唐诗代表了古代诗歌的最高成就，涌现出了李白、杜甫、王维等一大批杰出的诗人。他们的诗歌作品不仅具有极高的艺术价值，也反映了当时社会的风貌和人民的生活。

艺术方面，唐朝的绘画、雕塑、音乐、舞蹈等都达到了极高的水平。阎立本、吴道子等画家的作品至今仍被视为艺术瑰宝。唐朝的乐舞也十分发达，著名的《霓裳羽衣舞》便是其中的代表。

此外，唐朝的宗教文化也十分繁荣，佛教、道教、伊斯兰教等都在这一时期得到了广泛传播和发展。

**四、对外开放与外交关系**

唐朝是一个对外开放的王朝，与世界各国保持着广泛的交往。唐朝的使者、商人、学者等频

### 使用`with`语句进行资源管理

在处理流式响应时，建议使用`with`语句来确保资源的正确释放，避免潜在的资源泄漏问题。

In [ ]:
with client.chat.completions.create(
    model="glm-4-plus",
    messages=[
        {
            "role": "user",
            "content": "讲几个德国的有趣的知识",
        }
    ],
    stream=True
) as stream:
    for chunk in stream:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)

如果不使用`with`语句，则需要手动管理连接：

In [ ]:
stream = client.chat.completions.create(
    model="glm-4-plus",
    messages=[
        {
            "role": "user",
            "content": "有透明的金属吗？",
        }
    ],
    stream=True
)
try:
    for chunk in stream:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
finally:
    stream.close()
print("Exiting...")